<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-overview" data-toc-modified-id="Project-overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project overview</a></span></li><li><span><a href="#Scraping-otodom-link-for-offer-references" data-toc-modified-id="Scraping-otodom-link-for-offer-references-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scraping otodom link for offer references</a></span><ul class="toc-item"><li><span><a href="#Understanding-href-structure" data-toc-modified-id="Understanding-href-structure-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Understanding href structure</a></span></li><li><span><a href="#Extracting-all-offers-from-a-page" data-toc-modified-id="Extracting-all-offers-from-a-page-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Extracting all offers from a page</a></span></li><li><span><a href="#Extracting-all-following-pages" data-toc-modified-id="Extracting-all-following-pages-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Extracting all following pages</a></span></li><li><span><a href="#Create-a-list-of-all-offers" data-toc-modified-id="Create-a-list-of-all-offers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a list of all offers</a></span></li></ul></li><li><span><a href="#Extract-data-from-single-offer" data-toc-modified-id="Extract-data-from-single-offer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract data from single offer</a></span><ul class="toc-item"><li><span><a href="#Investigate-structure-of-single-offer-page" data-toc-modified-id="Investigate-structure-of-single-offer-page-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Investigate structure of single offer page</a></span></li><li><span><a href="#Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries" data-toc-modified-id="Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Drill down to data stored in json and strat exploring key dictionaries</a></span></li><li><span><a href="#Extract-key-details-into-clear-dictionary" data-toc-modified-id="Extract-key-details-into-clear-dictionary-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Extract key details into clear dictionary</a></span></li></ul></li><li><span><a href="#Combine-offers-into-json-list" data-toc-modified-id="Combine-offers-into-json-list-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combine offers into json list</a></span><ul class="toc-item"><li><span><a href="#Combine-offers-data-as-single-json-list" data-toc-modified-id="Combine-offers-data-as-single-json-list-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Combine offers data as single json list</a></span></li><li><span><a href="#Save-data-to-local-file-as-JSON-list-and-extract-it-from-local-drive" data-toc-modified-id="Save-data-to-local-file-as-JSON-list-and-extract-it-from-local-drive-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Save data to local file as JSON list and extract it from local drive</a></span></li></ul></li><li><span><a href="#Transforming-JSON-data-to-DataFrame" data-toc-modified-id="Transforming-JSON-data-to-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Transforming JSON data to DataFrame</a></span><ul class="toc-item"><li><span><a href="#Parse-parameters-data" data-toc-modified-id="Parse-parameters-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Parse parameters data</a></span></li><li><span><a href="#Parse-location-data" data-toc-modified-id="Parse-location-data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Parse location data</a></span></li><li><span><a href="#Parse-characteristics-data" data-toc-modified-id="Parse-characteristics-data-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Parse characteristics data</a></span></li></ul></li><li><span><a href="#Extracting-information-from-lists-and-transorming-data" data-toc-modified-id="Extracting-information-from-lists-and-transorming-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extracting information from lists and transorming data</a></span><ul class="toc-item"><li><span><a href="#Splitting-features-into-categories" data-toc-modified-id="Splitting-features-into-categories-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Splitting features into categories</a></span></li><li><span><a href="#Categorical-Pipeline" data-toc-modified-id="Categorical-Pipeline-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Categorical Pipeline</a></span><ul class="toc-item"><li><span><a href="#Dealing-with-Null-Values" data-toc-modified-id="Dealing-with-Null-Values-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Dealing with Null Values</a></span></li><li><span><a href="#Unique-categories" data-toc-modified-id="Unique-categories-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Unique categories</a></span></li><li><span><a href="#Multilabel-Features" data-toc-modified-id="Multilabel-Features-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Multilabel Features</a></span></li></ul></li><li><span><a href="#Numeric-pipeline" data-toc-modified-id="Numeric-pipeline-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Numeric pipeline</a></span></li><li><span><a href="#Combining-processed-data-into-final-dataframe" data-toc-modified-id="Combining-processed-data-into-final-dataframe-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Combining processed data into final dataframe</a></span></li><li><span><a href="#Transforming-string-data-about-floor-to-int" data-toc-modified-id="Transforming-string-data-about-floor-to-int-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Transforming string data about floor to int</a></span></li><li><span><a href="#Dealing-with-null-build-year-values---consider-if-the-building-is-under-counstruction" data-toc-modified-id="Dealing-with-null-build-year-values---consider-if-the-building-is-under-counstruction-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>Dealing with null build year values - consider if the building is under counstruction</a></span></li><li><span><a href="#Dealing-with-remaining-nulls-for-numerical-values-by-imputing-them-with-median" data-toc-modified-id="Dealing-with-remaining-nulls-for-numerical-values-by-imputing-them-with-median-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>Dealing with remaining nulls for numerical values by imputing them with median</a></span></li></ul></li><li><span><a href="#Generalizing-disctrict-categories-and-handling-misslabeling" data-toc-modified-id="Generalizing-disctrict-categories-and-handling-misslabeling-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Generalizing disctrict categories and handling misslabeling</a></span><ul class="toc-item"><li><span><a href="#Grouping-subdistricts-into-15-key-districts" data-toc-modified-id="Grouping-subdistricts-into-15-key-districts-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Grouping subdistricts into 15 key districts</a></span></li><li><span><a href="#Sanity-check-for-district-mapping" data-toc-modified-id="Sanity-check-for-district-mapping-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Sanity check for district mapping</a></span></li><li><span><a href="#Correcting-mislabelled-districts-by-nearest-neighbours-vote" data-toc-modified-id="Correcting-mislabelled-districts-by-nearest-neighbours-vote-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Correcting mislabelled districts by nearest neighbours vote</a></span></li></ul></li></ul></div>

In [106]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime
from scipy import spatial
import random
from collections import Counter

In [2]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

In [88]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

#Selecting a central city point to center all graphs around - Swietokrzyska Subway 
center_coors=52.235176, 21.008393

# Project overview
<a id="project_overview"><a>

The aim of this project is to combine webscraping practice with analysis of Real Estate prices in Warsaw. Creating an accurate price predictioin model requires combining multiple data sources such as RE offers from otodom, google drivetime data from googlemaps or data about reaturants and places of culture. 

- Part 1 focuses on websraping, data cleaning and transformation. One of most interesting elements of this notebook is imputation of misslabeled district names with use of nearest neighbour voting. 

- Part 2 deals with geospatial data vizualization.  I decided to experiment with geojson heatmaps to create immersive maps. 

- Part 3 is where I build several models to model predict and benchmark prices. Thorough error analysis based on spatial properties allows to balance model tuning with data enrichment as after initial training some of the errors were geogrphy specific, wchich means that our data did not capture some of the important location features. 

# Scraping otodom link for offer references


In [463]:
link="https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26"

In [4]:
html = urlopen(link)
bs = BeautifulSoup(html, 'lxml')

## Understanding href structure
<a id="href_struct"><a>

In [5]:
# Looking for references to tell apart offer data from adverts etc.
for item in bs.findAll("a",href=True):
    print(item)
    print("\n\n")

<a class="navbar-brand" href="https://www.otodom.pl" title="Otodom. Najpopularniejszy serwis nieruchomości">www.otodom.pl</a>



<a aria-expanded="false" class="dropdown-toggle" data-toggle="dropdown" data-tracking="classifieds_click" href="https://www.otodom.pl/sprzedaz/" role="button">Ogłoszenia <span class="caret"></span></a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"sale_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/sprzedaz/">Na sprzedaż</a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"rent_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/wynajem/">Na wynajem</a>



<a data-observed-action="" data-tracking="developers_click" data-tracking-data='{"touch_point_button":"investments_dropdown", "touch_point_page": "ads"}' href="https://www.otodom.pl/inwestycje/home/">Inwestycje</a>



<a aria-expanded="false" class="dropdown

It seems all offer hrefs contain "www.otodom.pl/oferta/" in their adress. 

The links to move to next page of search results contain "page=" label

In [6]:
bs.findAll("a",href=True)[1].attrs.keys()

dict_keys(['href', 'class', 'data-toggle', 'role', 'aria-expanded', 'data-tracking'])

## Extracting all offers from a page


In [7]:
for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/"))[:10]:
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/oferta/59m2-wola-jana-kazimierza-bezposrednio-ID45Qh0.html#8e0a67b43c


https://www.otodom.pl/oferta/59m2-wola-jana-kazimierza-bezposrednio-ID45Qh0.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-w-kamienicy-z-miejscami-postojowymi-ID436OC.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-w-kamienicy-z-miejscami-postojowymi-ID436OC.html#8e0a67b43c


https://www.otodom.pl/oferta/7pietro-10-200zl-m2-klucze-w-czerwcu-9min-metro-ID45Bp3.html#8e0a67b43c


https://www.otodom.pl/oferta/7pietro-10-200zl-m2-klucze-w-czerwcu-9min-metro-ID45Bp3.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-nowe-wykonczone-3-pok-0-pcc-garaz-ID45TXb.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-nowe-wykonczone-3-pok-0-pcc-garaz-ID45TXb.html#8e0a67b43c


https://www.otodom.pl/oferta/lux-apartament-w-kamienicy-bezposrednio-ID45KTI.html#8e0a67b43c


https://www.otodom.pl/oferta/lux-apartament-w-kamienicy-bezposrednio-ID45KTI.html#8e0a67b43c




In [8]:
#Function creating a list of offers hrefs from a single page

def get_page_offers(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    offer_list=[]

    for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/")):
        offer_list.append(item.attrs["href"])
    offer_list=list(set(offer_list))
    return(offer_list)

## Extracting all following pages


In [9]:
for item in bs.findAll("a", href=re.compile("page=")):
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=3


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=500


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2




Link to the next page change only number at the end of the url, there are maximum 500 pages in otodom - let's create list of 500 link to access all of them to scrap offers. 

In [468]:
page_max=500
base_next_page=link[:-1]
page_list=[link]
for i in range(2,page_max+1):

    new_page=base_next_page+str(i)
    page_list.append(new_page)

In [469]:
len(page_list)

500

## Create a list of all offers 


In [52]:
# Extracting link to each offer for each page in page_list

all_offers=[]
for i in range(0,len(page_list)-1):

    all_offers=all_offers+get_page_offers(page_list[i])
    if i%50==0:
        print("Scraping offers from link")
        print(page_list[i])
        print("Combined number of offers={}\n".format(len(all_offers)))
    
        

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26
Combined number of offers=30

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=51
Combined number of offers=1530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=101
Combined number of offers=3030

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=151
Combined number of offers=4530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=201
Combined number of offers=6030

Scraping offers from link
https://www

In [53]:
len(all_offers)

14969

# Extract data from single offer

## Investigate structure of single offer page 

In [54]:
offer=all_offers[10]

In [55]:
html = urlopen(offer)
bs = BeautifulSoup(html, 'lxml')

In [56]:
bs.title

<title data-react-helmet="true">5 pokoje, mieszkanie na sprzedaż - Warszawa, Mokotów - 60144854 • www.otodom.pl</title>

In [57]:
print(bs.prettify())

<!DOCTYPE html>
<html lang="pl">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="DCr06kqT-gkViKqUArVuJkKTZZ_RDpjNdUtk" name="csrf-token"/>
  <title data-react-helmet="true">
   5 pokoje, mieszkanie na sprzedaż - Warszawa, Mokotów - 60144854 • www.otodom.pl
  </title>
  <meta content="pl" data-react-helmet="true" http-equiv="Content-Language"/>
  <meta content="index, follow" data-react-helmet="true" name="robots"/>
  <meta content="Świetne 5 pokojowe mieszkanie na sprzedaż w miejscowości Warszawa, Mokotów, za cenę 3 500 000 zł. To mieszkanie na sprzedaż  na 3 piętrze ma 162,20 m² powierzchni użytkowej i 162,20 m² powierzchni całkowitej. Biuro nieruchomości POGORZELSKI NIERUCHOMOŚCI jako najważniejsze zalety mieszkania wymienia: garaż/miejsce parkingowe, ogródek, winda. Otodom 60144854" data-react-hel

Afte investigating, where the specific offer price could be found, we can see that it appears most times within sript with "id"= "server-app-state". Let's parse page html to find this script. 

In [58]:
data_script=bs.find("script",{"id":"server-app-state"})

In [59]:
data_script

<script id="server-app-state" type="application/json">{"initialProps":{"meta":{"target":{"Area":"162.2","AreaRange":[],"Build_year":"2001","Building_floors_num":"4","Building_material":["other"],"Building_ownership":["full_ownership"],"Building_type":["apartment"],"City":"warszawa","City_id":"26","Construction_status":["ready_to_use"],"Country":"Polska","Extras_types":["garage","garden","lift","terrace","usable_room"],"Floor_no":["floor_3"],"Heating":["urban"],"Id":"60144854","MarketType":"secondary","Media_types":["cable-television","internet"],"OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":3500000,"PriceRange":["3000001-10000000"],"Price_per_m":21578,"ProperType":"mieszkanie","Province":"ma

In [60]:
data_script.contents[0]

'{"initialProps":{"meta":{"target":{"Area":"162.2","AreaRange":[],"Build_year":"2001","Building_floors_num":"4","Building_material":["other"],"Building_ownership":["full_ownership"],"Building_type":["apartment"],"City":"warszawa","City_id":"26","Construction_status":["ready_to_use"],"Country":"Polska","Extras_types":["garage","garden","lift","terrace","usable_room"],"Floor_no":["floor_3"],"Heating":["urban"],"Id":"60144854","MarketType":"secondary","Media_types":["cable-television","internet"],"OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":3500000,"PriceRange":["3000001-10000000"],"Price_per_m":21578,"ProperType":"mieszkanie","Province":"mazowieckie","RegularUser":"n","Rooms_num":["5"],"Secur

## Drill down to data stored in json and strat exploring key dictionaries

In [61]:
data_json=json.loads(data_script.contents[0])

In [62]:
data_json

{'initialProps': {'meta': {'target': {'Area': '162.2',
    'AreaRange': [],
    'Build_year': '2001',
    'Building_floors_num': '4',
    'Building_material': ['other'],
    'Building_ownership': ['full_ownership'],
    'Building_type': ['apartment'],
    'City': 'warszawa',
    'City_id': '26',
    'Construction_status': ['ready_to_use'],
    'Country': 'Polska',
    'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
    'Floor_no': ['floor_3'],
    'Heating': ['urban'],
    'Id': '60144854',
    'MarketType': 'secondary',
    'Media_types': ['cable-television', 'internet'],
    'OfferType': 'sprzedaz',
    'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
    'Price': 3500000,
    'PriceRange': ['3000

After parsing json dictionary from the chosen sript, we can start to see clear patters. Let's focus on dictionary embedded within ['initialProps'], as it seems to have all the data we need to extract 

In [63]:
#parameters dict
data_json['initialProps']["meta"]["target"]

{'Area': '162.2',
 'AreaRange': [],
 'Build_year': '2001',
 'Building_floors_num': '4',
 'Building_material': ['other'],
 'Building_ownership': ['full_ownership'],
 'Building_type': ['apartment'],
 'City': 'warszawa',
 'City_id': '26',
 'Construction_status': ['ready_to_use'],
 'Country': 'Polska',
 'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
 'Floor_no': ['floor_3'],
 'Heating': ['urban'],
 'Id': '60144854',
 'MarketType': 'secondary',
 'Media_types': ['cable-television', 'internet'],
 'OfferType': 'sprzedaz',
 'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
 'Price': 3500000,
 'PriceRange': ['3000001-10000000'],
 'Price_per_m': 21578,
 'ProperType': 'mieszkanie',
 'Province': 'mazowieckie',
 

In [64]:
data_json['initialProps']["data"]["advert"]["price"]

{'human_value': '3 500 000 zł',
 'label': 'Cena',
 'unit': 'PLN',
 'value': '3500000'}

In [65]:
#Building characteristics data
data_json['initialProps']["data"]["advert"]['characteristics']

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '162.2',
  'value_translated': '162,20 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '5',
  'value_translated': '5'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'secondary',
  'value_translated': 'wtórny'},
 {'key': 'building_type',
  'currency': '',
  'label': 'Rodzaj zabudowy',
  'value': 'apartment',
  'value_translated': 'apartamentowiec'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_3',
  'value_translated': '3'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '4',
  'value_translated': '4'},
 {'key': 'building_material',
  'currency': '',
  'label': 'Materiał budynku',
  'value': 'other',
  'value_translated': 'inne'},
 {'key': 'heating',
  'currency': '',
  'label': 'Ogrzewanie',
  'value': 'urban',
  'value_translated': 'miejskie'},
 {'key': 'build_year',
  'currency': '',

In [66]:
# location data
data_json["initialProps"]["data"]["advert"]['location']

{'address': 'Warszawa, Mokotów, Zbyszka Cybulskiego',
 'coordinates': {'latitude': 52.19973,
  'longitude': 21.04167,
  'radius': 0,
  'zoom_level': 15},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '39', 'label': 'Mokotów', 'type': 'district'}]}

In [68]:
# Combining 3 key dictionaries containing property data into one json

offer_dict={}
offer_dict["params"]=data_json['initialProps']["meta"]["target"]
offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']

## Extract key details into clear dictionary

In [69]:
# Function extracting key data from each offer page

def get_offer_json(offer):
    html = urlopen(offer)
    bs = BeautifulSoup(html, 'lxml')
    data_script=bs.find("script",{"id":"server-app-state"})
    data_json=json.loads(data_script.contents[0])
    
    offer_dict={}
    offer_dict["params"]=data_json['initialProps']["meta"]["target"]
#    offer_dict["params"]["currency"]=data_json['initialProps']["data"]["advert"]["price"]["unit"]
    offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
    offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']
    
    return(offer_dict)

In [70]:
get_offer_json(offer)

{'params': {'Area': '162.2',
  'AreaRange': [],
  'Build_year': '2001',
  'Building_floors_num': '4',
  'Building_material': ['other'],
  'Building_ownership': ['full_ownership'],
  'Building_type': ['apartment'],
  'City': 'warszawa',
  'City_id': '26',
  'Construction_status': ['ready_to_use'],
  'Country': 'Polska',
  'Extras_types': ['garage', 'garden', 'lift', 'terrace', 'usable_room'],
  'Floor_no': ['floor_3'],
  'Heating': ['urban'],
  'Id': '60144854',
  'MarketType': 'secondary',
  'Media_types': ['cable-television', 'internet'],
  'OfferType': 'sprzedaz',
  'Photo': 'aHR0cHM6Ly9pcmVsYW5kLmFwb2xsby5vbHhjZG4uY29tL3YxL2ZpbGVzL2V5Sm1iaUk2SW0xblpuSXliMnQ2TkhrMU5ETXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLi0tN0hDZ0gtNHluYjdvVEpIdU5nVGFsRVRyQUJlbHBNVUxwVGsxeC1MQlEvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
  'Price': 3500000,
  'PriceRange': ['3000001-10000000'],
  'Price_per_m': 21578,
  'ProperType': 'mieszkani

# Combine offers into json list

## Combine offers data as single json list

In [71]:
#Function combining data from offer json, formatting it into a new dictionary. Creating a json list from all offers
def create_offers_dict(offers_list):
    combined_offers_list=[]
    for i in range(0, len(offers_list)):
        offer=offers_list[i]
        offer_data={}
        try:
            offer_json=get_offer_json(offer)
            offer_data["Id"]=offer_json["params"]["Id"]
            offer_data["date"]= str(datetime.utcnow().date())
            offer_data["data"]=offer_json
            combined_offers_list.append(offer_data)
        except:
            pass
      
        if i%500==0:
            print("Extracted {}/{} offers at {}".format(i,len(offers_list),str(datetime.now())[:-7]))
    return(combined_offers_list)
        
    

Let's extract all 15k offers - recommend leaving it running for the night as it might take a few hours. 

In [72]:
offers_json=create_offers_dict(all_offers)

Extracted 0/14969 offers at 2020-05-24 20:19:47
Extracted 500/14969 offers at 2020-05-24 20:26:58
Extracted 1000/14969 offers at 2020-05-24 20:34:04
Extracted 1500/14969 offers at 2020-05-24 20:41:33
Extracted 2000/14969 offers at 2020-05-24 20:49:04
Extracted 2500/14969 offers at 2020-05-24 20:56:25
Extracted 3000/14969 offers at 2020-05-24 21:03:56
Extracted 3500/14969 offers at 2020-05-24 21:11:20
Extracted 4000/14969 offers at 2020-05-24 21:18:53
Extracted 4500/14969 offers at 2020-05-24 21:26:15
Extracted 5000/14969 offers at 2020-05-24 21:33:31
Extracted 5500/14969 offers at 2020-05-24 21:41:14
Extracted 6000/14969 offers at 2020-05-24 21:48:30
Extracted 6500/14969 offers at 2020-05-24 21:55:48
Extracted 7000/14969 offers at 2020-05-24 22:02:49
Extracted 7500/14969 offers at 2020-05-24 22:10:06
Extracted 8000/14969 offers at 2020-05-24 22:17:28
Extracted 8500/14969 offers at 2020-05-24 22:24:30
Extracted 9000/14969 offers at 2020-05-24 22:31:23
Extracted 9500/14969 offers at 2020

In [73]:
len(offers_json)

14885

It seems that around 100 offers were not extracted correctly and were passed, but nearly 15k is a decent sample sufficient for further parts of the project. 

## Save data to local file as JSON list and extract it from local drive

In [74]:
#Saving list of jsons as a txt file

directory="./Warsaw_RE_data/"
if not os.path.exists(directory):
    os.makedirs(directory)
    
todaydate = str(datetime.utcnow().date())
file_name=directory+"Purchase_"+todaydate+".txt"
file_name

'./Warsaw_RE_data/Purchase_2020-05-24.txt'

In [75]:
text_file = open(file_name, "w")
text_file.write(json.dumps(offers_json))
text_file.close()

40788816

In [3]:
file_name="./Warsaw_RE_data/Purchase_2020-05-24.txt"

In [4]:
#Opening same file and converting it into a json list
json_list_txt = open(file_name, "r").read()
offers_json=json.loads(json_list_txt)

# Transforming JSON data to DataFrame

## Parse parameters data

In [5]:
#Transforming data stored under "params" key into a dataframe

def parse_param_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_row=json_list[i]["data"]["params"]
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')


    return(df_param)
  

In [6]:
df_params=parse_param_data(offers_json)
df_params.drop_duplicates(subset="Id",inplace=True)

In [7]:
df_params.head()

,Area,AreaRange,Build_year,Building_floors_num,City,City_id,Country,Extras_types,Floor_no,Id,...,State,Building_ownership,Building_type,Construction_status,Equipment_types,Media_types,Rent,Windows_type,Heating,Building_material
0,80.10,[],2021.0,8.0,warszawa,26,Polska,"[garage, lift, usable_room]",[floor_1],60534950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,warszawa,26,Polska,"[closed_area, elevator, garage, garden, terraces]",NaN,59816194,...,[in_building],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46.75,[],2019.0,6.0,warszawa,26,Polska,"[balcony, garage, lift, usable_room]",[floor_5],60534745,...,NaN,[full_ownership],[block],[ready_to_use],"[fridge, furniture, washing_machine]","[cable-television, internet]",440.0,[plastic],NaN,NaN
3,41.50,[],NaN,23.0,warszawa,26,Polska,[lift],[floor_higher_10],60534948,...,NaN,[co_operative_ownership_with_a_land_and_mortga...,[block],[ready_to_use],"[dishwasher, fridge, furniture, oven, stove, t...",[cable-television],350.0,NaN,[urban],NaN
4,102.00,[],2010.0,3.0,warszawa,26,Polska,"[balcony, garage, two_storey]",[floor_3],60477253,...,NaN,[full_ownership],[apartment],[ready_to_use],"[dishwasher, fridge, furniture, oven, stove, w...","[cable-television, internet, phone]",700.0,[plastic],[urban],[brick]


In [8]:
df_params.shape

(12089, 43)

## Parse location data

In [9]:
json_str=offers_json[0]["data"]["location"]

In [10]:
json_str

{'address': 'Warszawa, Wola, Juliana Konstantego Ordona',
 'coordinates': {'latitude': 52.22124,
  'longitude': 20.95634,
  'radius': 0,
  'zoom_level': 15},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '117', 'label': 'Wola', 'type': 'district'}]}

In [11]:
type(json_str["geoLevel"])==list

True

In [12]:
json_str.keys()

dict_keys(['address', 'coordinates', 'geoLevel'])

In [13]:
json_str["coordinates"]['latitude']

52.22124

In [14]:
#Transforming data stored under "location" into a single level dictionary

def extract_location_data(json_entry):
    new_dict={}
    json_str=json_entry["data"]["location"]
    for key in json_str.keys():
        val_level1=json_str[key]
        if type(val_level1)==dict:
            for subkey in val_level1.keys():
                val_level2=val_level1[subkey]
                new_dict[subkey]=val_level2

        elif type(val_level1)==list:
            for i in range(0, len(val_level1)):
                val_level2=val_level1[i]
                new_dict[val_level2["type"]]=val_level2["label"]
        else:

            new_dict[key]=json_str[key]
            
    return(new_dict)


In [15]:
extract_location_data(offers_json[0])

{'address': 'Warszawa, Wola, Juliana Konstantego Ordona',
 'latitude': 52.22124,
 'longitude': 20.95634,
 'radius': 0,
 'zoom_level': 15,
 'region': 'mazowieckie',
 'sub-region': 'Warszawa',
 'city': 'Warszawa',
 'district': 'Wola'}

In [16]:
#Transforming reformated location dictionary to DaraFrame

def parse_location_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_Id=json_list[i]["Id"]
        parsed_row=extract_location_data(json_list[i])
        parsed_row["Id"]=parsed_Id
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
       
    return(df_param)
  

In [17]:
df_location=parse_location_data(offers_json)
df_location.drop_duplicates(subset="Id",inplace=True)

In [18]:
df_location.head()

,address,latitude,longitude,radius,zoom_level,region,sub-region,city,district,Id,
0,"Warszawa, Wola, Juliana Konstantego Ordona",52.221240,20.956340,0,15,mazowieckie,Warszawa,Warszawa,Wola,60534950,NaN
1,"Warszawa, Targówek, Bródno, Bolesławicka 5",52.282996,21.031165,0,15,mazowieckie,Warszawa,Warszawa,Bródno,59816194,NaN
2,"Warszawa, Ursus, Ursus Faktory 2",52.197936,20.886419,0,14,mazowieckie,Warszawa,Warszawa,Ursus,60534745,NaN
3,"Warszawa, Centrum, ul. Świętokrzyska",52.235366,21.009301,0,16,mazowieckie,Warszawa,Warszawa,Centrum,60534948,NaN
4,"Warszawa, Wesoła, Stara Miłosna",52.219170,21.235200,100,14,mazowieckie,Warszawa,Warszawa,Stara Miłosna,60477253,NaN


In [19]:
df_location.columns

Index(['address', 'latitude', 'longitude', 'radius', 'zoom_level', 'region',
       'sub-region', 'city', 'district', 'Id', ''],
      dtype='object')

In [20]:
df_location.shape

(12089, 11)

## Parse characteristics data

In [21]:
json_str=offers_json[0]["data"]["characteristic"]

In [22]:
json_str

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '80.1',
  'value_translated': '80,10 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '4',
  'value_translated': '4'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'primary',
  'value_translated': 'pierwotny'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_1',
  'value_translated': '1'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '8',
  'value_translated': '8'},
 {'key': 'build_year',
  'currency': '',
  'label': 'Rok budowy',
  'value': '2021',
  'value_translated': '2021'},
 {'key': 'free_from',
  'currency': '',
  'label': 'Dostępne od',
  'value': '2021-12-31',
  'value_translated': '2021-12-31'}]

In [23]:
#Transforming reformated location dictionary to DaraFrame

def parse_characteristic_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_id=json_list[i]["Id"]
        extracted_dict={}
        parsed_dict=json_list[i]["data"]["characteristic"]
        for j in range(0, len(parsed_dict)):
            parsed_row=parsed_dict[j]
            extracted_dict[parsed_row["key"]]=parsed_row["value"]
        
        extracted_dict["Id"]=parsed_id
            
            
        
        param_list.append(extracted_dict)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    df_param.build_year=np.where(df_param.build_year>1700,df_param.build_year,np.NaN )
    df_param.build_year=np.where(df_param.build_year<2023,df_param.build_year,np.NaN )
    return(df_param)
  

In [24]:
df_characteristic=parse_characteristic_data(offers_json)
df_characteristic.drop_duplicates(subset="Id",inplace=True)

In [25]:
df_characteristic.columns

Index(['m', 'rooms_num', 'market', 'floor_no', 'building_floors_num',
       'build_year', 'free_from', 'Id', 'area_from', 'area_to',
       'price_per_m_from', 'price_from', 'price_to', 'floors_num', 'state',
       'offered_estates_type', 'building_type', 'windows_type',
       'construction_status', 'rent', 'building_ownership', 'heating',
       'building_material'],
      dtype='object')

In [26]:
df_characteristic.head()

,m,rooms_num,market,floor_no,building_floors_num,build_year,free_from,Id,area_from,area_to,...,floors_num,state,offered_estates_type,building_type,windows_type,construction_status,rent,building_ownership,heating,building_material
0,80.10,4,primary,floor_1,8.0,2021.0,2021-12-31,60534950,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59816194,38.0,84.0,...,4.0,in_building,flats,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46.75,3,secondary,floor_5,6.0,2019.0,NaN,60534745,NaN,NaN,...,NaN,NaN,NaN,block,plastic,ready_to_use,440.0,full_ownership,NaN,NaN
3,41.50,1,primary,floor_higher_10,23.0,NaN,NaN,60534948,NaN,NaN,...,NaN,NaN,NaN,block,NaN,ready_to_use,350.0,co_operative_ownership_with_a_land_and_mortgag...,urban,NaN
4,102.00,4,secondary,floor_3,3.0,2010.0,NaN,60477253,NaN,NaN,...,NaN,NaN,NaN,apartment,plastic,ready_to_use,700.0,full_ownership,urban,brick


In [27]:
df_characteristic.shape

(12089, 23)

In [28]:
#Saving 3 key dataframes locally in case they are needed later

df_params.to_excel("params.xlsx")
df_location.to_excel("location.xlsx")
df_characteristic.to_excel("characteristic.xlsx")

# Extracting information from lists and transorming data

## Splitting features into categories

In [29]:
#Combing 3 dataframes

df=pd.merge(df_params,df_location,left_on="Id",right_on="Id")
df=pd.merge(df,df_characteristic,left_on="Id",right_on="Id")
df.drop(columns={""},inplace=True)

In [30]:
df=df[df.Price.isna()==False]
df.reset_index(inplace=True)

In [31]:
df.shape

(11939, 75)

In [32]:
df.to_excel("combined_data.xlsx")

In [33]:
df.columns

Index(['index', 'Area', 'AreaRange', 'Build_year', 'Building_floors_num',
       'City', 'City_id', 'Country', 'Extras_types', 'Floor_no', 'Id',
       'MarketType', 'OfferType', 'Photo', 'Price', 'PriceRange',
       'Price_per_m', 'ProperType', 'Province', 'RegularUser', 'Rooms_num',
       'Security_types', 'Subregion', 'Title', 'env', 'seller_id', 'user_type',
       'Area_from', 'Area_to', 'Floors_num', 'Offered_estates_type',
       'Price_from', 'Price_per_m_from', 'Price_to', 'State',
       'Building_ownership', 'Building_type', 'Construction_status',
       'Equipment_types', 'Media_types', 'Rent', 'Windows_type', 'Heating',
       'Building_material', 'address', 'latitude', 'longitude', 'radius',
       'zoom_level', 'region', 'sub-region', 'city', 'district', 'm',
       'rooms_num', 'market', 'floor_no', 'building_floors_num', 'build_year',
       'free_from', 'area_from', 'area_to', 'price_per_m_from', 'price_from',
       'price_to', 'floors_num', 'state', 'offered_estat

In [34]:
#Splitting features in numeric, categorical with unique values and categorical with multiple values, as each type will be process differently

num_features=["Id","Area","Price","latitude","longitude","build_year","building_floors_num","rooms_num",
]
unique_features=["City","district","market","Building_material","Building_ownership","Building_type","Construction_status","floor_no","Heating",
"Windows_type"]
multi_features=["Equipment_types","Extras_types","Media_types","Security_types"]
cat_features=unique_features+multi_features

## Categorical Pipeline

### Dealing with Null Values

In [35]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin

In [36]:
class UniqueCatImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Replace null values with "not_specified" entry

        """ 

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        imp_fill_na = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value="not_specified")
        X=pd.DataFrame(imp_fill_na.fit_transform(X),columns=X.columns)
        
            
        return X



In [37]:
class MultiCatImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Replace null values with blank entry - it will be ommited during later transformation

        """ 

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        imp_fill_na = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value="")
        X=pd.DataFrame(imp_fill_na.fit_transform(X),columns=X.columns)
        
            
        return X



In [38]:
# Testing imputers before implementing them into Pipeline
imp_unique = UniqueCatImputer()
imp_multi = MultiCatImputer()


df[unique_features]=imp_unique .fit_transform(df[unique_features])
df[multi_features]=imp_multi.fit_transform(df[multi_features])

### Unique categories

In [39]:
from sklearn.base import BaseEstimator, TransformerMixin

In [40]:
class UniqueAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Extract unique values from list 

        """ 

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for feature in X.columns:
            X[feature]=X[feature].apply(lambda x:list_transform(x))
            
        
            
        return X



In [41]:
# Function used as lambda to change column type from list with len=1 to str

def list_transform(x):
    if type(x)==list:
        return(x[0])
    else:
        return(x)

In [42]:
unique_adder=UniqueAttributesAdder()
df_unique=unique_adder.fit_transform(df[unique_features])
df_unique.head()

,City,district,market,Building_material,Building_ownership,Building_type,Construction_status,floor_no,Heating,Windows_type
0,warszawa,Wola,primary,not_specified,not_specified,not_specified,not_specified,floor_1,not_specified,not_specified
1,warszawa,Ursus,secondary,not_specified,full_ownership,block,ready_to_use,floor_5,not_specified,plastic
2,warszawa,Centrum,primary,not_specified,co_operative_ownership_with_a_land_and_mortgag...,block,ready_to_use,floor_higher_10,urban,not_specified
3,warszawa,Stara Miłosna,secondary,brick,full_ownership,apartment,ready_to_use,floor_3,urban,plastic
4,warszawa,Żoliborz,secondary,not_specified,not_specified,block,to_renovation,floor_2,urban,not_specified


### Multilabel Features

In [43]:
mlb = MultiLabelBinarizer()

In [44]:
class MultiAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Extract data from columns containing lists with multiple categories, save them as one-hot columns.
        Append initial column prefix, to make multi features convention clear and avoid duplicates. 

        """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        features=X.columns
        for feature in features:
            feature_one_hot=mlb.fit_transform(X[feature])
            feature_labels=str(feature)+"_"+mlb.classes_
            df_one_hot_feature=pd.DataFrame(feature_one_hot, columns=feature_labels)
            if feature==features[0]:
                df_features=df_one_hot_feature
            else:
                df_features=pd.concat([df_features, df_one_hot_feature], axis=1, sort=False)
        
    
        
       
        
        return(df_features)

In [45]:
multi_adder=MultiAttributesAdder()
df_multi=multi_adder.fit_transform(df[multi_features])
df_multi.head()

,Equipment_types_dishwasher,Equipment_types_fridge,Equipment_types_furniture,Equipment_types_oven,Equipment_types_stove,Equipment_types_tv,Equipment_types_washing_machine,Extras_types_air_conditioning,Extras_types_attic,Extras_types_balcony,...,Media_types_internet,Media_types_phone,Media_types_sewage,Media_types_water,Security_types_alarm,Security_types_anti_burglary_door,Security_types_closed_area,Security_types_entryphone,Security_types_monitoring,Security_types_roller_shutters
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,1,0,0,0,1,0,0,1,...,1,0,0,0,0,1,0,1,0,0
2,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1,1,1,1,0,1,0,0,1,...,1,1,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0


## Numeric pipeline

In [46]:
class NumTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        """
        Return numeric values without transformation, possibly changed to more advanced numeric pipleine in future releases. 

        """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        
    
        
       
        
        return(X)

## Combining processed data into final dataframe

In [47]:
# Single dataframe processing all chosen features. Developed based on code build in 6.2-6.3

from sklearn.pipeline import Pipeline
unique_cat_pipeline = Pipeline([
    ("imputer", UniqueCatImputer()),
    ("attr_adder", UniqueAttributesAdder()),
    ])

multi_cat_pipeline = Pipeline([
    ("imputer", MultiCatImputer()),
    ("attr_adder", MultiAttributesAdder()),
    ])

num_pipeline = Pipeline([
    ("transformer",NumTransformer())
])

In [48]:
df_num=num_pipeline.fit_transform(df[num_features])
df_cat_unique=unique_cat_pipeline.fit_transform(df[unique_features])
df_cat_multi=multi_cat_pipeline.fit_transform(df[multi_features])

In [49]:
df_num.shape

(11939, 8)

In [50]:
df_cat_unique.shape

(11939, 10)

In [51]:
df_cat_multi.shape

(11939, 30)

In [189]:
df_fin=pd.concat([df_num, df_cat_unique, df_cat_multi], sort=False, axis=1)

## Transforming string data about floor to int

"Floor_no" is a feature, which seems categorical, but should be changed to int. As there are only few categories available it will be easiest to do it with a dictionary mapping all the names in a logical manner. 

In [190]:
for i in df_fin["floor_no"].unique():
    print(i)

floor_1
floor_5
floor_higher_10
floor_3
floor_2
floor_10
floor_4
ground_floor
floor_6
floor_9
not_specified
floor_8
floor_7
cellar
garret


In [191]:
floor_dict={
    "floor_1":1,
    "floor_2":2,
    "floor_3":3,
    "floor_4":4,
    "floor_5":5,
    "floor_6":6,
    "floor_7":7,
    "floor_8":8,
    "floor_9":9,
    "floor_10":10,
    "floor_higher_10":11,
    "ground_floor":0,
    "not_specified":np.NaN,
    "cellar":-1,
    "garret":12,
    
}

In [192]:
df_fin["floor_num"]=RE_prices_transformed["floor_no"].apply(lambda x:floor_dict[x])

## Dealing with null build year values - consider if the building is under counstruction

In [193]:
# Let's assume that if building status is "in completion", it will probably be finished no earlier than 2021

df_fin["build_year"]=np.where((df_fin.build_year.isna()) & (df_fin.Construction_status=="to_completion"),2021,df_fin.build_year )

In [194]:
from sklearn.impute import SimpleImputer
imp_const = SimpleImputer(missing_values=np.nan, strategy='median')

## Dealing with remaining nulls for numerical values by imputing them with median

In [195]:
null_features=["floor_num","build_year","building_floors_num"]

In [196]:
df_fin[null_features]=imp_const.fit_transform(df_fin[null_features])

In [197]:
df_fin.isna().sum()

Id                                   0
Area                                 0
Price                                0
latitude                             0
longitude                            0
build_year                           0
building_floors_num                  0
rooms_num                            0
City                                 0
district                             0
market                               0
Building_material                    0
Building_ownership                   0
Building_type                        0
Construction_status                  0
floor_no                             0
Heating                              0
Windows_type                         0
Equipment_types_dishwasher           0
Equipment_types_fridge               0
Equipment_types_furniture            0
Equipment_types_oven                 0
Equipment_types_stove                0
Equipment_types_tv                   0
Equipment_types_washing_machine      0
Extras_types_air_conditio

# Generalizing disctrict categories and handling misslabeling

## Grouping subdistricts into 15 key districts

In [198]:
df_fin.rename(columns={"district":"subdistrict"},inplace=True)

In [199]:
district_mapping={
    
    'Wola':"Wola", 
    'Młynów':"Wola",
    'Powązki':"Wola",
    'Czyste':"Wola", 
    'Ulrychów':"Wola", 
    'Koło':"Wola",
    'Odolany':"Wola", 


    

    
    'Marysin Wawerski':"Wawer",
    'Sadul':"Wawer", 
    'Wawer':"Wawer",
    'Zerzeń':"Wawer",
    'Anin':"Wawer",
    'Gocławek':"Wawer",
    


    
    'Bemowo':"Bemowo", 
    'Nowe Górce':"Bemowo", 
    'Jelonki':"Bemowo", 
    'Groty':"Bemowo",
    'Chrzanów':"Bemowo", 
    "Górce":"Bemowo",
    



    
    'Ochota':"Ochota", 
    'Stara Ochota':"Ochota",
    'Szczęśliwice':"Ochota",
    'Rakowiec':"Ochota", 
    'Filtry':"Ochota",
    'Okęcie':"Ochota",
    'Raków':"Ochota",
    
    


    'Praga-Północ':"Praga",
    'Nowa Praga':"Praga",
    'Praga':"Praga",

    'Pelcowizna':"Praga",
    'Stara Praga':"Praga",

    
    'Praga-Południe':"Southern Praga",
    'Saska Kępa':"Southern Praga",
    'Kamionek':"Southern Praga",
    'Witolin':"Southern Praga",
    'Gocław':"Southern Praga",
    'Grochów':"Southern Praga",  
    
    

    
    'Żerań':"Targowek",
    'Targówek':"Targowek",
    'Bródno':"Targowek", 
    'Szmulowizna':"Targowek",
    'Targówek Mieszkaniowy':"Targowek",
    'Nowe Bródno':"Targowek", 
    'Bródno-Podgrodzie':"Targowek",
    'Targówek Fabryczny':"Targowek",
    'Zacisze':"Targowek", 
    
    
    

    

    
    'Wilanów':"Wilanow",  
    'Zawady':"Wilanow",
    'Powsinek':"Wilanow", 
    'Powsin':"Wilanow", 



    'Mokotów':"Mokotow",
    'Górny Mokotów':"Mokotow", 
    'Dolny Mokotów':"Mokotow",
    'Pola Mokotowskie':"Mokotow",
    'Wyględów':"Mokotow",
    'Sielce':"Mokotow", 
    'Czerniaków':"Mokotow", 
    'Służewiec':"Mokotow",
    'Wierzbno':"Mokotow",
    'Potok':"Mokotow", 
    'Sadyba':"Mokotow", 
    'Służew':"Mokotow", 
    'Ksawerów':"Mokotow",
    'Metro Wilanowska':"Mokotow",
    'Augustówek':"Mokotow",
    'Siekierki':"Mokotow",
    'Królikarnia':"Mokotow", 
    'Stegny':"Mokotow",
    'Zielona':"Mokotow", 
    



    
    
    'Śródmieście Północne':"Downtown",
    'Śródmieście Południowe':"Downtown",
    'Mirów':"Downtown",
    'Centrum':"Downtown",
    'Śródmieście':"Downtown", 
    'Stare Miasto':"Downtown", 
    'Mariensztat':"Downtown",
    'Nowe Miasto':"Downtown",
    'Muranów':"Downtown",
    'Powiśle':"Downtown", 
    
    
 
    

    'Białołęka':"Bialoleka",
    'Białołęka Dworska':"Bialoleka",
    'Nowodwory':"Bialoleka",
    'Tarchomin':"Bialoleka",
    'Brzeziny':"Bialoleka", 
    'Annopol':"Bialoleka", 
    'Dąbrówka Szlachecka':"Bialoleka", 
    'Winnica':"Bialoleka",
    'Płudy':"Bialoleka",
    
    'Włochy':"Wlochy",
    'Nowe Włochy':"Wlochy",
    'Stare Włochy':"Wlochy",



    
    'Żoliborz':"Zoliborz",
    'Sady Żoliborskie':"Zoliborz",
    'Stary Żoliborz':"Zoliborz",
    'Marymont':"Zoliborz",
    'Cytadela':"Zoliborz",

    
    
    'Bielany':"Bielany", 
    'Młociny':"Bielany",
    'Słodowiec':"Bielany",
    'Chomiczówka':"Bielany",
    'Wawrzyszew':"Bielany", 
    'Piaski':"Bielany",
    'Wrzeciono':"Bielany", 
    'Stare Bielany':"Bielany", 
    

    'Natolin':"Ursynow",
    'Wyczółki':"Ursynow", 
    'Imielin':"Ursynow",
    'Kabaty':"Ursynow",
    'Stokłosy':"Ursynow",
    'Grabów':"Ursynow", 
    'Stary Imielin':"Ursynow",  
    'Ursynów':"Ursynow",
    "Pyry":"Ursynow",
    


    'Lewandów':"Subburbs", 
    'Wola Grzybowska':"Subburbs", 
    'Kąty Grodziskie':"Subburbs",
    'Stary Rembertów':"Subburbs",
    'Nowy Rembertów':"Subburbs",
    'Olszynka Grochowska':"Subburbs", 
    'Gołąbki':"Subburbs", 
    'Salomea':"Subburbs", 
    'Międzylesie':"Subburbs",
    'Zbytki':"Subburbs",
    'Nowe Jeziorki':"Subburbs", 
    'Radość':"Subburbs", 
    'Falenica':"Subburbs",
    'Rembertów':"Subburbs",
    'Grodzisk':"Subburbs",
    'Ursus':"Subburbs", 
    'Skorosze':"Subburbs", 
    'Wesoła':"Subburbs", 
    'Stara Miłosna':"Subburbs",



    'not_specified':"Other", 
    
    

    

}

In [200]:
df_fin["district"]=df_fin.subdistrict.apply(lambda x:district_mapping[x])

In [201]:
df_fin[["district","Id"]].groupby("district",as_index=False).count().sort_values(by="Id",ascending=False)

,district,Id
4,Mokotow,1765
15,Wola,1415
3,Downtown,1307
1,Bialoleka,1225
8,Southern Praga,991
9,Subburbs,779
0,Bemowo,692
11,Ursynow,566
5,Ochota,472
13,Wilanow,469


Now the samples look much better - grouped Districts should make it easier to determine if any of them affect the price or is it determined purely by geodata such as drivetime. 

## Sanity check for district mapping

In [202]:
data=[]

for dist in df_fin.district.unique():
    df_temp=df_fin[df_fin.district==dist]
    trace=go.Scattermapbox(
                lat=df_temp.latitude,
                lon=df_temp.longitude,

                mode='markers',
                marker=dict(
                    opacity=0.8, 
                    size=10, 

                   ),
                    text=df_temp.district,
                    name=dist,


                )
    data.append(trace)




layout = go.Layout(
    title="Key districts of Warsaw",  
    title_x=0.4,
   
    
    
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 80, b = 0, l = 0, r = 0),
    font = dict(color = 'dark grey', size = 18),
 
    mapbox = dict(
       

        center = dict(
            lat = center_coors[0],
            lon = center_coors[1]
        ),
        # default level of zoom
        zoom = 10,
        # default map style
        style = "carto-positron"
    )

)

figure=go.Figure(
    data=data,
    layout=layout,
  
    )



iplot(figure)


This map looks quite messy - there are a lot of mislabeled points, far outside of the actual District boundaries. Let's adjust the labels, by selecting k nearest neighbours and selecting the district by a majority vote of these labels. 

## Correcting mislabelled districts by nearest neighbours vote

In [203]:
coors_array=np.ones([df_fin.shape[0],2])
noise=np.random.rand(df_fin.shape[0],2)/10000

In [204]:
coors_array[:,0]=df_fin.latitude
coors_array[:,1]=df_fin.longitude

In [205]:
coors_array=coors_array+noise

In [206]:
tree = spatial.KDTree(coors_array)

In [207]:
n_votes=7
neighbours_distance,neighbours_idx= tree.query(coors_array,k=n_votes)

In [208]:
neighbours_idx

array([[    0,  5159,  2701, ...,  5439, 10471,   688],
       [    1,  7942,  1534, ...,  5269,  2886, 11266],
       [    2,  1873,  8552, ...,  5281,  9979, 11881],
       ...,
       [11936,  4935,  8203, ..., 10256, 11799,  9127],
       [11937, 11164,  6235, ...,  5399,  6446,  1340],
       [11938,  1076,  7698, ...,  4905,  8767, 11151]])

In [209]:
district_vote_list=[]

for i in range(0,df_fin.shape[0]):
    dist_list=[]
    for j in range(0,n_votes):
        nearest_dist=df_fin.iloc[neighbours_idx[i,j]].district
        dist_list.append(nearest_dist)
    
    data = Counter(dist_list)
    district_vote= data.most_common()[0][0]
    if district_vote=="Other":
        district_vote= data.most_common()[1][0]
        
    district_vote_list.append(district_vote)

district_vote_arr=np.asarray(district_vote_list)

In [210]:
df_fin["district_mod"]=district_vote_arr

In [211]:
data=[]

for dist in df_fin.district_mod.unique():
    df_temp=df_fin[df_fin.district_mod==dist]
    trace=go.Scattermapbox(
                lat=df_temp.latitude,
                lon=df_temp.longitude,

                mode='markers',
                marker=dict(
                    opacity=0.5, 
                    size=10, 

                   ),
                    text=df_temp.district_mod,
                    name=dist,


                )
    data.append(trace)




layout = go.Layout(
    title="Key districts of Warsaw",  
    title_x=0.4,
   
    
    
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 80, b = 0, l = 0, r = 0),
    font = dict(color = 'dark grey', size = 18),
 
    mapbox = dict(
       

        center = dict(
            lat = center_coors[0],
            lon = center_coors[1]
        ),
        # default level of zoom
        zoom = 10,
        # default map style
        style = "carto-positron"
    )

)

figure=go.Figure(
    data=data,
    layout=layout,
  
    )



iplot(figure)


This map looks much neatier - there are still a few outliers remaining, but the noise from mislabelling seems to be much visible now. We also managed to get rid of all "Not Specified" entries, which were Null before, by assigning them the most frequent district of their neighbours. 

In [212]:
# Replacing district with the cleaned data using nearest neighbours voting

df_fin.rename(columns={"district":"district_old","district_mod":"district"},inplace=True)

In [213]:
df_fin.columns

Index(['Id', 'Area', 'Price', 'latitude', 'longitude', 'build_year',
       'building_floors_num', 'rooms_num', 'City', 'subdistrict', 'market',
       'Building_material', 'Building_ownership', 'Building_type',
       'Construction_status', 'floor_no', 'Heating', 'Windows_type',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_oven',
       'Equipment_types_stove', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_attic', 'Extras_types_balcony', 'Extras_types_basement',
       'Extras_types_garage', 'Extras_types_garden', 'Extras_types_lift',
       'Extras_types_separate_kitchen', 'Extras_types_terrace',
       'Extras_types_two_storey', 'Extras_types_usable_room',
       'Media_types_cable-television', 'Media_types_electricity',
       'Media_types_internet', 'Media_types_phone', 'Media_types_sewage',
       'Media_types_water', 'Security_types_alarm

In [215]:
#Saving to local file for further processing

df_fin.to_excel("Warsaw_RE_data.xlsx",index=False)